# Clasificación de Ingresos con Machine Learning para Análisis Socioeconómico

En este proyecto se desarrolla un sistema de **clasificación de ingresos personales** basado en modelos de *machine learning*, cuyo objetivo es predecir si un individuo gana **más o menos de \$50K anuales** utilizando datos demográficos y laborales extraídos del censo.

El sistema está diseñado para analizar registros individuales y, mediante un algoritmo de **clasificación supervisada**, identificar patrones socioeconómicos relevantes que puedan ser útiles en estudios de mercado, políticas públicas, o decisiones comerciales.

Adicionalmente, se contempla la posibilidad de extender el análisis para generar **visualizaciones interactivas** o aplicar técnicas de segmentación poblacional con fines estadísticos o estratégicos.

## Objetivo

- Desarrollar un modelo capaz de **predecir el nivel de ingresos** de una persona a partir de datos censales.
- Aplicar técnicas de *preprocesamiento*, *entrenamiento* y *evaluación* de modelos de clasificación binaria.
- Diseñar un pipeline reproducible que permita aplicar el modelo en nuevos conjuntos de datos.

## Alcance del Proyecto

- Exploración de datos (EDA) y visualización de variables clave.
- Entrenamiento y validación de modelos de clasificación (por ejemplo: Regresión Logística, Árboles de Decisión, Random Forest).
- Evaluación de métricas como *accuracy*, *precision*, *recall*, y *f1-score*.
- Identificación de las variables más relevantes mediante técnicas de importancia de características.

## Uso Comercial

Este tipo de análisis tiene múltiples aplicaciones prácticas en sectores como:

- **Banca y Finanzas**: Evaluación crediticia y segmentación de clientes.
- **Marketing**: Diseño de campañas según el perfil socioeconómico.
- **Gobierno y ONGs**: Políticas de inclusión y estudios poblacionales.

## Dataset Utilizado: Adult Census Income

Se utiliza el *dataset* **Adult Income (también conocido como Census Income)**, extraído del Censo de EE. UU. de 1994 y preparado por Ronny Kohavi y Barry Becker para tareas de minería de datos.

Características principales del *dataset*:

- **Número de instancias**: ~48,000 registros.
- **Atributos**: 14 variables incluyendo edad, educación, ocupación, horas trabajadas por semana, entre otros.
- **Variable objetivo**: `income`, que indica si el individuo gana `>50K` o `<=50K` al año.
- **Tipo de datos**: Mixto (categóricos y numéricos).
- **Fuente original**: Base de datos de la Oficina del Censo de los Estados Unidos.


> **Nota sobre la limpieza del dataset**:
> 
> Antes de incluir los registros en el conjunto de datos final, se aplicaron ciertos **filtros de limpieza** para asegurar que los datos sean relevantes y de calidad. Estos filtros fueron:
>
> - `AAGE > 16`: Se excluyen personas menores de 17 años, ya que generalmente no participan activamente en el mercado laboral.
> - `AGI > 100`: Aunque `AGI` no figura directamente en la versión final, este filtro garantiza que se consideren solo registros con ingresos válidos y no triviales.
> - `AFNLWGT > 1`: Se descartan registros con peso muestral casi nulo. El campo `fnlwgt` indica cuántas personas en la población representa ese registro. Si es muy bajo, el dato tiene poca relevancia estadística.
> - `HRSWK > 0`: Se omiten personas que no trabajan nada en la semana, ya que no aportan información útil al análisis de ingresos.
>
> Estos filtros ayudan a mantener un conjunto de datos limpio, útil y representativo para entrenar modelos de clasificación.


### Acceso al Dataset

El *dataset* puede descargarse desde Kaggle:

👉 [Adult Census Income Dataset en Kaggle](https://www.kaggle.com/datasets/uciml/adult-census-income)

Esta versión puede contener columnas mejor etiquetadas y separadas para análisis más avanzados o uso con frameworks como **TensorFlow**, **PyTorch** o **XGBoost**.

### Basado en investigaciones académicas

Este proyecto toma como base el trabajo académico publicado por:

> **Ron Kohavi**, “Scaling Up the Accuracy of Naive-Bayes Classifiers: a Decision-Tree Hybrid”, *KDD 1996*.

Además, es ampliamente utilizado como benchmark en cursos, certificaciones y desafíos de ciencia de datos debido a su simplicidad y valor práctico.

Para este trabajo se utilizará la versión en Kaggle, por su formato limpio y accesible.

## EDA: Exploratory Data Analysis

El primer paso en este proyecto es realizar un análisis exploratorio de datos (EDA) sobre el *dataset* **Adult Census Income**. Este proceso es clave para entender la naturaleza de los datos disponibles y preparar el terreno para la fase de modelado.

A diferencia de otros *datasets* integrados en librerías como `scikit-learn`, el *Adult Income* proviene de fuentes externas (como Kaggle o UCI), y se presenta habitualmente en formato **CSV**. Por lo tanto, podemos utilizar directamente funciones como `head()`, `info()` y `describe()` de **pandas** para comenzar la exploración.

> **Nota**: Antes de aplicar modelos de machine learning, es esencial entender la distribución de los datos, la presencia de valores nulos, y las relaciones entre las variables.

### Descripción del Dataset

El *dataset* Adult Income contiene información demográfica y laboral de individuos adultos residentes en EE.UU., recolectada por la Oficina del Censo. El objetivo es predecir si el ingreso anual de una persona supera los **\$50,000**.

Una vez cargado en un `DataFrame` de `pandas`, el conjunto de datos se verá de la siguiente manera:

| Columna             | Tipo       | Descripción                                                             |
|---------------------|------------|-------------------------------------------------------------------------|
| `age`               | Numérica   | Edad del individuo.                                                    |
| `workclass`         | Categórica | Tipo de empleo (privado, gobierno, autónomo, etc.).                    |
| `fnlwgt`            | Numérica   | Peso muestral (indica cuántas personas representa esta muestra).       |
| `education`         | Categórica | Nivel educativo (HS-grad, Bachelors, etc.).                            |
| `education-num`     | Numérica   | Nivel educativo en formato numérico.                                   |
| `marital-status`    | Categórica | Estado civil.                                                           |
| `occupation`        | Categórica | Ocupación laboral.                                                     |
| `relationship`      | Categórica | Relación familiar (esposo/a, hijo/a, etc.).                            |
| `race`              | Categórica | Raza declarada.                                                        |
| `sex`               | Categórica | Género (Male/Female).                                                  |
| `capital-gain`      | Numérica   | Ganancias de capital obtenidas.                                        |
| `capital-loss`      | Numérica   | Pérdidas de capital registradas.                                       |
| `hours-per-week`    | Numérica   | Cantidad de horas trabajadas por semana.                               |
| `native-country`    | Categórica | País de origen.                                                        |
| `income`            | Categórica | Variable objetivo: `>50K` o `<=50K` (clase a predecir).                |

> **Nota**: Algunas columnas como `education` y `education-num` son redundantes, pero pueden utilizarse para contrastar codificaciones.

### Carga de Datos

Los datos serán descargados directamente desde Kaggle mediante el siguiente código:

In [3]:
# Se importa la librería 'kagglehub' para permitir la descarga de datasets desde Kaggle.
import kagglehub  

# Se descarga la última versión disponible del dataset "adult-census-income" desde el repositorio "uciml" en Kaggle.
path = kagglehub.dataset_download("uciml/adult-census-income")  

# Se imprime la ruta local en la que se han almacenado los archivos descargados del dataset.
print("Path to dataset files:", path)  

Path to dataset files: C:\Users\George\.cache\kagglehub\datasets\uciml\adult-census-income\versions\3
